In [24]:
import pandas as pd
import folium
from ipywidgets import interact
import ipywidgets as widgets

# 자전거 데이터 불러오기
y_pm_1 = pd.read_csv('y_pm_1.csv')  # 1월 자전거 데이터 파일 경로
y_pm_2 = pd.read_csv('y_pm_2.csv')  # 2월 자전거 데이터 파일 경로
y_pm_3 = pd.read_csv('y_pm_3.csv')  # 3월 자전거 데이터 파일 경로
y_pm_4 = pd.read_csv('y_pm_4.csv')  # 4월 자전거 데이터 파일 경로
y_pm_5 = pd.read_csv('y_pm_5.csv')  # 5월 자전거 데이터 파일 경로
y_pm_6 = pd.read_csv('y_pm_6.csv')  # 6월 자전거 데이터 파일 경로

# 지하철 데이터 불러오기
y_sub_1 = pd.read_csv('y_sub_1.csv')  # 1월 지하철 데이터 파일 경로
y_sub_2 = pd.read_csv('y_sub_2.csv')  # 2월 지하철 데이터 파일 경로
y_sub_3 = pd.read_csv('y_sub_3.csv')  # 3월 지하철 데이터 파일 경로
y_sub_4 = pd.read_csv('y_sub_4.csv')  # 4월 지하철 데이터 파일 경로
y_sub_5 = pd.read_csv('y_sub_5.csv')  # 5월 지하철 데이터 파일 경로
y_sub_6 = pd.read_csv('y_sub_6.csv')  # 6월 지하철 데이터 파일 경로

# 전체 데이터를 담을 딕셔너리 생성
bike_monthly_data = {
    '1월': y_pm_1,
    '2월': y_pm_2,
    '3월': y_pm_3,
    '4월': y_pm_4,
    '5월': y_pm_5,
    '6월': y_pm_6
}

subway_monthly_data = {
    '1월': y_sub_1,
    '2월': y_sub_2,
    '3월': y_sub_3,
    '4월': y_sub_4,
    '5월': y_sub_5,
    '6월': y_sub_6
}

def show_monthly_data(month, data_type):
    # 서울 중심부에 지도 생성
    seoul_map = folium.Map(location=[37.52508926, 126.9240799], zoom_start=14)
    
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>공공자전거와 지하철의 이용합계 위치도</b></h3>
                 '''
    caption_html = '''
        <div style="
            position: fixed;
            bottom: 50px; 
            left: 50px;
            z-index: 1000;
            padding: 10px;
            background-color: white;
            border: 2px solid grey;
            font-size: 14px;
            ">
            <p><b>공공자전거 정류소 - 상위 3개:</b></p>
            <p>202번 국민일보 앞, 207번 여의나루역 1번출구 앞, 5854번 신한금융투자 앞</p>
            <p><b>특징:</b></p>
            <p>이용량 높은 공공자전거 정류소는 고정.</p>
            <p>여의도 외곽보다 중심부에 위치.</p>
            <p>일반상업지역으로 지정.</p>
            <p>마포대교와 인접 혹은 이에 따라 생성된 대로변에 위치.</p>
            <br>
            <p><b>지하철 역 - 상위 1개:</b></p>
            <p>여의도 역</p>
            <p><b>특징:</b></p>
            <p>여의도공원 및 여의도의 중심부에 위치.</p>
            <p>인근에 증권 관련 기업과 기관 많음.</p>
            <p>5호선과 9호선이 교차하는 교차역이자 환승역.</p>
            <br>
            <p><b>공통 특징:</b></p>
            <p>모두 인구 수요를 발생시키는 요건에 해당하는 것.</p>
            <p>날이 비교적 풀리는 3월부터 야외 활동이 증가함에 따라 이용량 증가.</p>
            <br>
            <p><b>*주의사항:</b></p>
            <p>데이터 버튼을 빠르게 누를 시, 오류 발생. 3초 정도의 간격을 두고 누를 시 오류 발생 x.</p>
        </div>
    '''
    
    seoul_map.get_root().html.add_child(folium.Element(title_html))  # 대시보드에 제목 추가
    seoul_map.get_root().html.add_child(folium.Element(caption_html))  # 대시보드에 캡션 추가
    
    if data_type == '자전거':
        df = bike_monthly_data[month]  # 선택한 월에 해당하는 자전거 데이터 가져오기
        
        # 자전거 데이터 처리
        df['대여건수'] = df['대여건수'].str.replace(',', '').astype(int)
        df['반납건수'] = df['반납건수'].str.replace(',', '').astype(int)
        df['대여반납건수_합계'] = df['대여건수'] + df['반납건수']

        # 기존의 색상 구분 방식 유지
        top_3 = df.nlargest(3, '대여반납건수_합계')
        top_3_names = top_3['대여소명'].tolist()
        
        top_11 = df.nlargest(11, '대여반납건수_합계')[3:]
        top_11_values = top_11['대여반납건수_합계'].tolist()
        min_value = min(top_11_values)
        max_value = max(top_11_values)
        
        for index, row in df.iterrows():
            if row['대여소명'] in top_3_names:
                color = 'red'
            else:
                normalized_value = (row['대여반납건수_합계'] - min_value) / (max_value - min_value)
                if normalized_value <= 0.125:
                    color = '#FFA500'
                elif 0.125 < normalized_value <= 0.25:
                    color = '#FFD700'
                elif 0.25 < normalized_value <= 0.375:
                    color = '#FFEC8B'
                elif 0.375 < normalized_value <= 0.5:
                    color = '#ADFF2F'
                else:
                    color = '#7FFF00'

            tooltip_text = f"{row['대여소명']}: {row['대여반납건수_합계']} 건"
            folium.CircleMarker([row['X'], row['Y']], radius=5, tooltip=tooltip_text, fill=True, color=color, fill_color=color).add_to(seoul_map)
    
    elif data_type == '지하철':
        df = subway_monthly_data[month]  # 선택한 월에 해당하는 지하철 데이터 가져오기
        
        # 지하철 데이터 처리
        df['승차승객수'] = df['승차승객수'].str.replace(',', '').astype(int)
        df['하차승객수'] = df['하차승객수'].str.replace(',', '').astype(int)
        df['승하차승객수'] = df['승차승객수'] + df['하차승객수']

        # 기존의 색상 구분 방식 유지
        top_4 = df.nlargest(4, '승하차승객수')
        top_4_names = top_4['역명'].tolist()
        
        for index, row in df.iterrows():
            if row['역명'] == top_4_names[0]:
                color = 'red'
            elif row['역명'] in top_4_names[1:3]:
                color = '#FF8C00'
            else:
                color = '#006400'

            tooltip_text = f"{row['역명']}: {row['승하차승객수']} 명"
            folium.CircleMarker([row['x'], row['y']], radius=5, tooltip=tooltip_text, fill=True, color=color, fill_color=color).add_to(seoul_map)

    display(seoul_map)

# 월과 데이터 유형에 대한 드롭다운 메뉴 생성
months_dropdown = widgets.Dropdown(options=list(bike_monthly_data.keys()), description='월 선택:')
data_type_dropdown = widgets.Dropdown(options=['자전거', '지하철'], description='데이터 선택:')

# 상호작용 가능한 대시보드 생성
interact(show_monthly_data, month=months_dropdown, data_type=data_type_dropdown)

interactive(children=(Dropdown(description='월 선택:', options=('1월', '2월', '3월', '4월', '5월', '6월'), value='1월'),…

<function __main__.show_monthly_data(month, data_type)>